In [0]:
# Read all cleaned tables
dim_player = spark.read.option("header", True).csv("/mnt/mount2/dim_player")
dim_country = spark.read.option("header", True).csv("/mnt/mount2/dim_country")
dim_club = spark.read.option("header", True).csv("/mnt/mount2/dim_club")
fact_player = spark.read.option("header", True).csv("/mnt/mount2/fact_player_statistics")
dim_match_type = spark.read.option("header", True).csv("/mnt/mount2/dim_match_type")

In [0]:
# output mount point generation
configs = {
  "fs.azure.account.key.adlsstoragehockey.blob.core.windows.net": "clFz+IqyE3a2urx/PnxFLcfk8vwUclZTGvY9GcGjVzI+t2/d6RSpUjGqaHIhEUyahCPGtmF+d4zL+AStz2XpCQ=="
}
dbutils.fs.mount(
  source = "wasbs://report@adlsstoragehockey.blob.core.windows.net/",
  mount_point = "/mnt/mount3",
  extra_configs=configs
)

True

REPORT 1: Player Report with Country & Club Info


In [0]:
player_report = dim_player \
    .select("Player_ID", "Player Name", "Country_ID", "Club_ID") \
    .join(dim_country.select("Country_ID", "Country_Name"), on="Country_ID", how="left") \
    .join(dim_club.select("Club_ID", "Club_Name"), on="Club_ID", how="left") \
    .select("Player_ID", "Player Name", "Country_Name", "Club_Name")

player_report.write.option("header", True).mode("overwrite").csv("/mnt/mount3/player_country_club_report")

REPORT 2: Player with Max Appearances in International Matches

In [0]:
from pyspark.sql.functions import col, desc

In [0]:
# Filter International Matches
fact_with_match = fact_player \
    .join(dim_match_type.select("Match_Type_ID", "Match_Name"), on="Match_Type_ID", how="left") \
    .filter(col("Match_Name") == "International")

# Group and find max appearances
max_appearance = fact_with_match.groupBy("Player_ID").agg({"Appearances": "sum"}) \
    .withColumnRenamed("sum(Appearances)", "Total_Appearances")

# Get player with max appearances
from pyspark.sql.functions import max as max_
max_value = max_appearance.agg(max_("Total_Appearances")).first()[0]

top_players = max_appearance.filter(col("Total_Appearances") == max_value) \
    .join(dim_player.select("Player_ID", "Player Name"), on="Player_ID", how="left")

top_players.write.option("header", True).mode("overwrite").csv("/mnt/mount3/max_international_appearance")

REPORT 3: Top 10 Goal Scorers of 2010


In [0]:
dim_time = spark.read.option("header", True).csv("/mnt/mount2/dim_time")

In [0]:
# Filter fact_player for Year = 2010 and join with dim_player
fact_2010 = fact_player.join(dim_time.select("Time_ID", "Year"), on="Time_ID") \
                       .filter(col("Year") == "2010")

top_goal_scorers_2010 = fact_2010 \
    .join(dim_player.select("Player_ID", "Player Name"), on="Player_ID", how="left") \
    .select("Player_ID", "Player Name", col("Goals_Scored").cast("int")) \
    .orderBy(desc("Goals_Scored")) \
    .limit(10)

# Save to report container
top_goal_scorers_2010.write.option("header", True).mode("overwrite").csv("/mnt/mount3/top_10_goal_scorers_2010")

REPORT 4: Top 5 Players with Most Red Cards

In [0]:
top_red_card_players = fact_player \
    .join(dim_player.select("Player_ID", "Player Name"), on="Player_ID", how="left") \
    .select("Player_ID", "Player Name", col("Red_Card").cast("int")) \
    .orderBy(desc("Red_Card")) \
    .limit(5)

top_red_card_players.write.option("header", True).mode("overwrite").csv("/mnt/mount3/top_5_redcard_players")

REPORT 5: Top 5 Successful Clubs (by Wins)

In [0]:
fact_team = spark.read.option("header", True).csv("/mnt/mount2/fact_team_statistics")

In [0]:
top_clubs = fact_team \
    .join(dim_club.select("Club_ID", "Club_Name"), on="Club_ID", how="left") \
    .select("Club_Name", col("Wins").cast("int")) \
    .groupBy("Club_Name").sum("Wins") \
    .withColumnRenamed("sum(Wins)", "Total_Wins") \
    .orderBy(desc("Total_Wins")) \
    .limit(5)

top_clubs.write.option("header", True).mode("overwrite").csv("/mnt/mount3/top_5_successful_clubs")